<a href="https://colab.research.google.com/github/Latamila/PyTorch-e-Lightning/blob/main/LMM_Modelagem_Deploy_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Mini-Projeto 3 - Modelagem</font>
## <font color='blue'>Fine-Tuning de Modelo LLM Para Tarefa Específica e Deploy de Web App com Gradio</font>

![DSA](imagens/MP3.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


In [3]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [4]:
!pip install -q torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [5]:
!pip install -q transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.0 MB/s eta 0:00:00


In [6]:
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00


In [7]:
# Imports
import torch
import transformers
import numpy as np
import torch.nn.functional as F
from accelerate import Accelerator
from torch.utils.data import Dataset
from transformers import AutoConfig, AutoModelForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer

In [8]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

torch       : 2.0.1
numpy       : 1.25.2
transformers: 4.31.0



## Carregando o LLM

https://huggingface.co/gpt2

O modelo terá a mesma arquitetura do GPT-2, mas com algumas modificações para torná-lo menor. As principais mudanças são o tamanho do vocabulário que é 13 porque só vai lidar com números mais o padding token, o "+" e o "=". A janela de contexto suportará apenas 6 tokens, pois estamos interessados apenas em realizar a adição de dois dígitos únicos.

In [9]:
# Tamanho do vocabulário
vocab_size = 13

In [10]:
# Comprimento da sequência
sequence_length = 4

In [11]:
# Comprimento do resultado
result_length = 2

In [12]:
# Comprimento do contexto
context_length = sequence_length + result_length

In [13]:
# Parâmetros de configuração do modelo GPT-2
config = AutoConfig.from_pretrained("gpt2",
                                    vocab_size = vocab_size,
                                    n_ctx = context_length,
                                    n_head = 4,
                                    n_layer = 2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [14]:
# Carrega o modelo
modelo = AutoModelForCausalLM.from_config(config)

In [15]:
# Função para calcular o tamanho do modelo
def model_size(model):
    return sum(t.numel() for t in modelo.parameters())

In [16]:
print(f'Tamanho do Modelo: {model_size(modelo)/1000**2:.1f}M parâmetros')

Tamanho do Modelo: 15.0M parâmetros


Este modelo tem 15 milhões de parâmetros em vez dos 111 milhões de parâmetros da configuração padrão "gpt2".

In [17]:
type(modelo)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [18]:
# Salva o modelo em disco
modelo.save_pretrained("modelos/modelo_inicial")

## Tokenizador Personalizado

In [19]:
# Definindo uma classe chamada NumberTokenizer, que é usada para tokenizar os números
class DSATokenizer:

    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, numbers_qty = 10):

        # Lista de tokens possíveis que o tokenizador pode encontrar
        vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

        # Definindo a quantidade de números que o tokenizador pode lidar
        self.numbers_qty = numbers_qty

        # Definindo o token de preenchimento (padding)
        self.pad_token = '-1'

        # Criando um dicionário que mapeia cada token para um índice único
        self.encoder = {str(v):i for i,v in enumerate(vocab)}

        # Criando um dicionário que mapeia cada índice único de volta ao token correspondente
        self.decoder = {i:str(v) for i,v in enumerate(vocab)}

        # Obtendo o índice do token de preenchimento no encoder
        self.pad_token_id = self.encoder[self.pad_token]

    # Método para decodificar uma lista de IDs de token de volta para uma string
    def decode(self, token_ids):
        return ' '.join(self.decoder[t] for t in token_ids)

    # Método que é chamado quando o objeto da classe é invocado como uma função
    def __call__(self, text):
        # Dividindo o texto em tokens individuais e retornando uma lista dos IDs correspondentes
        return [self.encoder[t] for t in text.split()]

> Vamos testar o tokenizador!

In [20]:
# Cria o objeto do tokenizador
tokenizer = DSATokenizer(vocab_size)

In [21]:
# Decoder do tokenizador
tokenizer.decoder

{0: '+',
 1: '=',
 2: '-1',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

In [22]:
# Testando o tokenizador
tokenizer("1 + 1 = 2")

[4, 0, 4, 1, 5]

## Criando o Dataset

O conjunto de dados deve ser criado neste formato:

- Entrada: "2 + 3 = 0" onde os 4 primeiros caracteres representam a sequência de entrada e o quinto caractere representa o primeiro caracter da saída.

- Saída: "+ 3 = 0 5" onde os 2 últimos dígitos representam o resultado da adição e os 3 primeiros dígitos são ignorados durante o treinamento e preenchidos com o pad.

O resultado é um conjunto de dados de sequências tokenizadas de números.

In [23]:
# Definindo uma classe chamada CriaDataset, que herda da classe Dataset do PyTorch
class CriaDataset(Dataset):

    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, split, length = 6):

        # Verificando se a divisão do dataset (split) é 'treino' ou 'teste'
        assert split in {'treino', 'teste'}
        self.split = split
        self.length = length

    # Definindo o método len que retorna o tamanho do dataset.
    # Nesse caso, o tamanho é fixo e igual a 1 milhão.
    def __len__(self):
        return 1000000

    # Definindo o método getitem que é usado para obter um item específico do dataset
    def __getitem__(self, idx):

        # Criando uma lista com todos os números disponíveis que não são tokens de padding e são numéricos
        available_numbers = [int(n) for n in tokenizer.decoder.values() if n != tokenizer.pad_token and str(n).isnumeric()]

        # Selecionando aleatoriamente números da lista de números disponíveis para criar uma entrada (input)
        inp = torch.tensor(np.random.choice(available_numbers, size = result_length))

        # Calculando a soma dos números selecionados e criando um tensor
        sol = torch.tensor([int(i) for i in str(inp.sum().item())])

        # Preenchendo o tensor com zeros para que tenha o tamanho desejado
        sol = torch.nn.functional.pad(sol, (1 if sol.size()[0] == 1 else 0,0), 'constant', 0)

        # Concatenando a entrada e a solução em um tensor
        cat = torch.cat((inp, sol), dim = 0)

        # Criando os tensores de entrada e alvo para o treinamento do modelo
        x = cat[:-1].clone()
        y = cat[1:].clone()

        # Definindo o primeiro elemento do tensor alvo como o token de padding
        y[:1] = int(tokenizer.pad_token)

        # Transformando os tensores x e y em strings
        x = str(x[0].item()) + ' + ' + str(x[1].item()) + ' = ' + str(x[2].item())
        y = '-1 ' + str(y[0].item()) + ' -1 ' + str(y[1].item()) + ' ' + str(y[2].item())

        # Tokenizando as strings de entrada e alvo
        tokenized_input = tokenizer(x)
        tokenized_output = tokenizer(y)

        # Retornando os tensores de entrada e alvo como itens do dataset
        return torch.tensor(tokenized_input), torch.tensor(tokenized_output)

## Datasets de Treino e Teste

In [24]:
dataset_treino = CriaDataset('treino', length = sequence_length)

In [25]:
dataset_teste = CriaDataset('teste', length = sequence_length)

In [26]:
x, y = dataset_treino[0]

In [27]:
x

tensor([12,  0, 11,  1,  4])

In [28]:
y

tensor([ 2,  2,  2,  4, 10])

In [29]:
print(tokenizer.decode(x.numpy()))

9 + 8 = 1


In [30]:
print(tokenizer.decode(y.numpy()))

-1 -1 -1 1 7


## Loop de Treinamento

In [31]:
num_epochs = 2

In [32]:
batch_size = 100

In [33]:
optimizer = torch.optim.Adam(modelo.parameters())

In [34]:
dados = torch.utils.data.DataLoader(dataset_treino, shuffle = True, batch_size = batch_size)

https://pypi.org/project/accelerate/

In [35]:
import accelerate
from accelerate import Accelerator

In [36]:
accelerator = Accelerator()

In [37]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy       : 1.25.2
transformers: 4.31.0
accelerate  : 0.27.2
torch       : 2.0.1



In [38]:
modelo, optimizer, dados = accelerator.prepare(modelo, optimizer, dados)

In [39]:
modelo.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(13, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=13, bias=False)
)

In [47]:
# Iniciando o loop para as épocas de treinamento
for epoch in range(num_epochs):

    # Iterando por cada batch (conjunto) de dados de entrada e alvos no dataset de treinamento
    for source, targets in dados:

        # Resetando os gradientes acumulados no otimizador
        optimizer.zero_grad()

        # Calculando a perda (loss) através da entropia cruzada entre as previsões do modelo e os alvos verdadeiros.
        # Os tensores são "achatados" para que possam ser passados para a função de entropia cruzada.
        # O índice do token de preenchimento (pad_token) é ignorado no cálculo da perda.
        loss = F.cross_entropy(modelo(source).logits.flatten(end_dim = 1),
                               targets.flatten(end_dim = 1),
                               ignore_index = tokenizer.pad_token_id)

        # Calculando os gradientes da perda em relação aos parâmetros do modelo
        accelerator.backward(loss)

        # Atualizando os parâmetros do modelo utilizando os gradientes calculados
        optimizer.step()

        # Recalculando a perda após a etapa de otimização.
        loss = F.cross_entropy(modelo(source).logits.flatten(end_dim = 1),
                               targets.flatten(end_dim = 1),
                               ignore_index = tokenizer.pad_token_id)

    # Imprimindo a época atual e a perda após cada época de treinamento
    print(f'Epoch: {epoch+1}/{num_epochs} --- Erro: {loss.item()}')

Epoch: 1/2 --- Erro: 6.514172855531797e-05
Epoch: 2/2 --- Erro: 1.7583369071871857e-07


## Avaliação do Modelo

In [41]:
# Definindo a função gera_solution com três parâmetros: input, solution_length e model
def faz_previsao(entrada, solution_length = 6, model = modelo):

    # Colocando o modelo em modo de avaliação.
    model.eval()

    # Convertendo a entrada (string) em tensor utilizando o tokenizer.
    # O tensor é uma estrutura de dados que o modelo de aprendizado de máquina pode processar.
    entrada = torch.tensor(tokenizer(entrada))

    # Enviando o tensor de entrada para o dispositivo de cálculo disponível (CPU ou GPU)
    entrada = entrada.to(accelerator.device)

    # Iniciando uma lista vazia para armazenar a solução
    solution = []

    # Loop que gera a solução de comprimento solution_length
    for i in range(solution_length):

        # Alimentando a entrada atual ao modelo e obtendo a saída
        saida = model(entrada)

        # Pegando o índice do maior valor no último conjunto de logits (log-odds) da saída,
        # que é a previsão do modelo para o próximo token
        predicted = saida.logits[-1].argmax()

        # Concatenando a previsão atual com a entrada atual.
        # Isso servirá como a nova entrada para a próxima iteração.
        entrada = torch.cat((entrada, predicted.unsqueeze(0)), dim = 0)

        # Adicionando a previsão atual à lista de soluções e convertendo o tensor em um número Python padrão
        solution.append(predicted.cpu().item())

    # Decodificando a lista de soluções para obter a string de saída e retornando-a
    return tokenizer.decode(solution)

In [42]:
# Definindo a função avalia_modelo com dois parâmetros: num_samples e log
def avalia_modelo(num_samples = 1000, log = False):

    # Iniciando um contador para as previsões corretas
    correct = 0

    # Loop que itera num_samples vezes
    for i in range(num_samples):

        # Obtendo a entrada e o alvo (resposta correta) do i-ésimo exemplo do conjunto de teste
        entrada, target = dataset_teste[i]

        # Convertendo os tensores de entrada e alvo em arrays numpy para processamento posterior
        entrada = entrada.cpu().numpy()
        target = target.cpu().numpy()

        # Decodificando a entrada e o alvo utilizando o tokenizer
        entrada = tokenizer.decode(entrada[:sequence_length])
        target = tokenizer.decode(target[sequence_length-1:])

        # Gerando a previsão utilizando a função faz_previsao
        predicted = faz_previsao(entrada, solution_length = result_length, model = modelo)

        # Se a previsão for igual ao alvo, incrementa o contador de previsões corretas
        if target == predicted:
            correct += 1
            # Se log for True, imprime detalhes do exemplo e a previsão correta
            if log:
                print(f'Acerto do Modelo: Input: {entrada} Target: {target} Previsão: {predicted}')
        else:
            # Se log for True, imprime detalhes do exemplo e a previsão errada
            if log:
                print(f'Erro do Modelo: Input: {entrada} Target: {target} Previsão: {predicted}')

    # Ao final do loop, calcula a acurácia (número de previsões corretas dividido pelo número total de exemplos)
    print(f'Acurácia: {correct/num_samples}')

In [43]:
# Executa a função
avalia_modelo(num_samples = 10, log = True)

Acerto do Modelo: Input: 2 + 4 = Target: 0 6 Previsão: 0 6
Acerto do Modelo: Input: 6 + 0 = Target: 0 6 Previsão: 0 6
Acerto do Modelo: Input: 8 + 1 = Target: 0 9 Previsão: 0 9
Acerto do Modelo: Input: 3 + 0 = Target: 0 3 Previsão: 0 3
Acerto do Modelo: Input: 4 + 5 = Target: 0 9 Previsão: 0 9
Erro do Modelo: Input: 6 + 2 = Target: 0 8 Previsão: 0 9
Erro do Modelo: Input: 1 + 0 = Target: 0 1 Previsão: 0 2
Acerto do Modelo: Input: 8 + 1 = Target: 0 9 Previsão: 0 9
Erro do Modelo: Input: 7 + 0 = Target: 0 7 Previsão: 0 8
Erro do Modelo: Input: 6 + 6 = Target: 1 2 Previsão: 1 4
Acurácia: 0.6


In [44]:
# Executa a função
avalia_modelo(num_samples = 1000, log = False)

Acurácia: 0.623


In [45]:
type(modelo)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [46]:
modelo.save_pretrained("modelos/modelo_final")

# Fim

In [48]:
!pip install -q gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.1 MB/s eta 0:00:00


In [49]:
import torch
import gradio as gr
from transformers import AutoModelForCausalLM

In [50]:
%reload_ext watermark
%watermark -a 'Camila' --iversions

Author: Camila

gradio      : 4.19.2
numpy       : 1.25.2
transformers: 4.31.0
accelerate  : 0.27.2
torch       : 2.0.1



In [51]:
modelo_llm = AutoModelForCausalLM.from_pretrained('modelos/modelo_final')

In [52]:
# Definindo uma classe chamada NumberTokenizer, que é usada para tokenizar os números
class DSATokenizer:

    # Método construtor da classe, que é executado quando um objeto dessa classe é criado
    def __init__(self, numbers_qty = 10):

        # Lista de tokens possíveis que o tokenizador pode encontrar
        vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

        # Definindo a quantidade de números que o tokenizador pode lidar
        self.numbers_qty = numbers_qty

        # Definindo o token de preenchimento (padding)
        self.pad_token = '-1'

        # Criando um dicionário que mapeia cada token para um índice único
        self.encoder = {str(v):i for i,v in enumerate(vocab)}

        # Criando um dicionário que mapeia cada índice único de volta ao token correspondente
        self.decoder = {i:str(v) for i,v in enumerate(vocab)}

        # Obtendo o índice do token de preenchimento no encoder
        self.pad_token_id = self.encoder[self.pad_token]

    # Método para decodificar uma lista de IDs de token de volta para uma string
    def decode(self, token_ids):
        return ' '.join(self.decoder[t] for t in token_ids)

    # Método que é chamado quando o objeto da classe é invocado como uma função
    def __call__(self, text):
        # Dividindo o texto em tokens individuais e retornando uma lista dos IDs correspondentes
        return [self.encoder[t] for t in text.split()]

In [53]:
# Cria o objeto
tokenizer = DSATokenizer(13)

In [54]:
# Definindo a função gera_solution com três parâmetros: input, solution_length e model
def faz_previsao(entrada, solution_length = 6, model = modelo_llm):

    # Colocando o modelo em modo de avaliação.
    model.eval()

    # Convertendo a entrada (string) em tensor utilizando o tokenizer.
    # O tensor é uma estrutura de dados que o modelo de aprendizado de máquina pode processar.
    entrada = torch.tensor(tokenizer(entrada))

    # Iniciando uma lista vazia para armazenar a solução
    solution = []

    # Loop que gera a solução de comprimento solution_length
    for i in range(solution_length):

        # Alimentando a entrada atual ao modelo e obtendo a saída
        saida = model(entrada)

        # Pegando o índice do maior valor no último conjunto de logits (log-odds) da saída,
        # que é a previsão do modelo para o próximo token
        predicted = saida.logits[-1].argmax()

        # Concatenando a previsão atual com a entrada atual.
        # Isso servirá como a nova entrada para a próxima iteração.
        entrada = torch.cat((entrada, predicted.unsqueeze(0)), dim = 0)

        # Adicionando a previsão atual à lista de soluções e convertendo o tensor em um número Python padrão
        solution.append(predicted.cpu().item())

    # Decodificando a lista de soluções para obter a string de saída e retornando-a
    return tokenizer.decode(solution)

In [55]:
# Testa a função
faz_previsao('3 + 5 =', solution_length = 2)

'0 8'

In [56]:
# Função para retornar a função que faz a previsão
def funcsolve(entrada):
    return faz_previsao(entrada, solution_length = 2)

In [57]:
# Cria a web app
webapp = gr.Interface(fn = funcsolve,
                      inputs = [gr.Textbox(label = "Dados de Entrada",
                                           lines = 1,
                                           info = "Os dados devem estar na forma: '1 + 2 =' com um único espaço entre cada caractere e apenas números de um dígito são permitidos.")],
                      outputs = [gr.Textbox(label = "Resultado (Previsão do Modelo)", lines = 1)],
                      title = "Deploy de LLM Após o Fine-Tuning",
                      description = "Digite os dados de entrada e clique no botão Submit para o modelo fazer a previsão.",
                      examples = ["5 + 3 =", "2 + 9 ="])

In [58]:
webapp.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e909710a41f0851e17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


FIM